In [ ]:
# Создание обучающей выборки на основе панхроматического канала

import rasterio
from rasterio.mask import mask
import geopandas as gpd
from shapely.geometry import mapping

gdb_path = 'template_path' #Заменить на путь до шейп-файла
templates = gpd.read_file(gdb_path)

input_raster_path = 'raster_path' #Заменить на путь до исходного растра

result_path = 'result_path' #Заменить на директорию для хранения обучающей выборки. Предварительно создать папки roads и non_roads
for _, template in templates.iterrows():
    template_border = mapping(template['geometry'])
    output_path = ""

    if template['type'] == 1:
        output_path = rf"{result_path}\roads\road_{template['Id']}.tif"
    elif template['type'] == 2:
        output_path = rf"{result_path}\non_roads\non_road_{template['Id']}.tif"

    with rasterio.open(input_raster_path) as src:
        out_image, out_transform = mask(src, [template_border], crop=True)

        out_meta = src.meta.copy()
        out_meta.update({
            "driver": "GTiff",
            "height": out_image.shape[1],
            "width": out_image.shape[2],
            "transform": out_transform
        })

        with rasterio.open(output_path, "w", **out_meta) as dest:
            dest.write(out_image)

In [ ]:
# Для работы с трехканальными изображениями необходимо их предварительно перевести в формат jpg
import os
import rasterio
from rasterio.mask import mask
import geopandas as gpd
from shapely.geometry import mapping
from PIL import Image, ImageOps
import numpy as np

input_raster_path = 'raster_path' #Заменить на путь до исходного растра
result_path_jpg = 'result_path_jpg' #Заменить на директорию для хранения файла jpg

with rasterio.open(input_raster_path) as src:
    out_image = src.read()
    out_image = (out_image * (255 / np.max(out_image))).astype(np.uint8)
    out_image_pil = Image.fromarray(np.moveaxis(out_image, 0, -1), 'RGBA')
    out_image_pil = out_image_pil.convert('RGB')
    out_image_pil = ImageOps.autocontrast(out_image_pil)
    out_image_pil.save(output_path, 'JPEG')

#Создание патчей
gdb_path = 'template_path' #Заменить на путь до шейп-файла
templates = gpd.read_file(gdb_path)

input_raster_path = 'raster_path' #Заменить на путь до исходного растра

result_path = 'result_path' #Заменить на директорию для хранения обучающей выборки. Предварительно создать папки roads и non_roads

for _, template in templates.iterrows():
    template_border = mapping(template['geometry'])
    output_path = ""

    if template['type'] == 1:
        output_path = rf"{result_path}\roads\road_{template['Id']}.jpg" 
    elif template['type'] == 2:
        output_path = rf"{result_path}\non_roads\non_road_{template['Id']}.jpg"

    print(f"Обработка шаблона ID {template['Id']}...")

    try:
        with rasterio.open(input_raster_path) as src:
            out_image, out_transform = mask(src, [template_border], crop=True)
            out_image = (out_image * (255 / np.max(out_image))).astype(np.uint8)
            out_image_pil = Image.fromarray(np.moveaxis(out_image, 0, -1), 'RGBA')

            out_image_pil = out_image_pil.convert('RGB') #Данные WorldView-2 были получены в виде RGBA. Если альфа-канала 

            out_image_pil = ImageOps.autocontrast(out_image_pil)  # Применяем автоконтраст

            # Сохраняем в формате JPG
            out_image_pil.save(output_path, 'JPEG') #Сохраняем изображение

    except Exception as e:
        print(f"Ошибка при обработке шаблона {template['Id']}: {e}")

print("Обрезка завершена!")